In [6]:
import os
import sys
src_path = os.path.split(os.getcwd())[0]
sys.path.insert(0, src_path)
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from clip import helpers

def morgan_from_smiles(smiles, radius=3, nbits=1024, chiral=True):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=nbits, useChirality=chiral)
    arr = np.zeros((0,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp,arr)
    return arr


if __name__ == '__main__':
    #indir = "<path-to-your-folder>"
    #index = "<path-to-your-folder>/cellpainting-index.csv"
    #index = os.path.join(indir, index)
 #   index = '/share/data/analyses/silvija/RT/data_cloome/our_images/metadata_try_v5.csv'
 #   index = '/share/data/analyses/silvija/RT/data_cloome/our_images/metadata_P102785.csv'          # Used for 6 samples
    index = '/share/data/analyses/silvija/RT/data_cloome/our_images/preprocessing_all/metadata_P102785.csv'          # preprocessing_all/ added
    
    #outdir = "/publicwork/sanchez/data/"
    outdir = '/share/data/analyses/silvija/RT/data_cloome/our_images/preprocessing_all/'           # preprocessing_all/ added
#    outfile_hdf = "morgan_chiral_fps.hdf5"                                                        # Used for 6 samples
    outfile_hdf = "morgan_chiral_fps_P102785.hdf5"
    outfile_hdf = os.path.join(outdir, outfile_hdf)

    n_cpus = 60

    csv = pd.read_csv(index)

  #  print(csv)

    csv["ID"] = csv.apply(lambda row: "-".join([str(row["PLATE_ID"]), str(row["WELL_POSITION"]),  str(row["SITE"])]), axis=1)
    
    ids = csv["ID"]
    smiles = csv["SMILES"]
    
    fps = helpers.parallelize(morgan_from_smiles, smiles, n_cpus)

    columns = [str(i) for i in range(fps[0].shape[0])]

    df = pd.DataFrame(fps, index=ids, columns=columns)

    df["SAMPLE_KEY"] = df.index
   # df = pd.DataFrame(fps, index=ids, columns=columns)
   # df["SAMPLE_KEY"] = ids
    
    df.to_hdf(outfile_hdf, key="df", mode="w")

In [11]:
# Loading of the created HDF5 file
#hdf5_filename = '/share/data/analyses/silvija/RT/data_cloome/our_images/morgan_chiral_fps.hdf5'
hdf5_filename = outfile_hdf

df_loaded = pd.read_hdf(hdf5_filename, key='df')

print(f"Shape of dataframe : {df_loaded.shape}\n")
print(df_loaded.head())


Shape of dataframe : (2385, 1025)

               0  1  2  3  4  5  6  7  8  9  ...  1015  1016  1017  1018  \
ID                                           ...                           
P102785-A03-1  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
P102785-A03-2  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
P102785-A03-3  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
P102785-A03-4  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
P102785-A03-5  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   

               1019  1020  1021  1022  1023     SAMPLE_KEY  
ID                                                          
P102785-A03-1     1     0     0     0     0  P102785-A03-1  
P102785-A03-2     1     0     0     0     0  P102785-A03-2  
P102785-A03-3     1     0     0     0     0  P102785-A03-3  
P102785-A03-4     1     0     0     0     0  P102785-A03-4  
P102785-A03-5     1     0     0     0     0  P102785-A03-5  

[5 r

In [3]:
print(df_loaded[df_loaded['SAMPLE_KEY'] == 'P102785-K09-2'])

               0  1  2  3  4  5  6  7  8  9  ...  1015  1016  1017  1018  \
ID                                           ...                           
P102785-K09-2  0  1  1  0  1  0  0  0  0  0  ...     0     0     0     0   

               1019  1020  1021  1022  1023     SAMPLE_KEY  
ID                                                          
P102785-K09-2     1     1     0     0     0  P102785-K09-2  

[1 rows x 1025 columns]


In [4]:
duplicate_sample_keys = df_loaded[df_loaded.duplicated(subset='SAMPLE_KEY', keep=False)]['SAMPLE_KEY'].unique()
print(duplicate_sample_keys)

[]


In [5]:
df_mol = pd.read_csv('/share/data/analyses/silvija/official_information_annotations_and_SMILES.xlsx - Chemical information.csv')
print(df_mol[df_mol['Compound ID'] == 'CBK289990'])

     Compound ID   Batch nr CUSTOMER_ID       MOLFORMULA  MOLWEIGHT  \
579    CBK289990  BJ1896664   A54141700         C20H24N2     292.43   
4592   CBK289990  BJ1894466   K83192048  C20H24N2.C4H4O4     408.50   

                      NAME        VENDOR                           ADD_INFO  \
579            dimetindene  BOC Sciences                      CAS 5636-83-9   
4592  dimethindene-(S)-(+)    Bio-Techne  CAS 136152-65-3, maleic acid salt   

                                             SMILES  \
579             CC(C1=C(CCN(C)C)Cc2ccccc12)c1ccccn1   
4592  C[C@H](C3=NC=CC=C3)C2=C(CCN(C)C)CC1=CC=CC=C12   

                                             IUPAC_NAME STEREOCHEMISTRY  \
579   N,N-dimethyl-2-[3-[1-(2-pyridyl)ethyl]-1H-inde...             NaN   
4592  N,N-Dimethyl-3-[(1S)-1-(2-pyridinyl)ethyl]-1H-...     (1S)-isomer   

                pert_iname clinical_phase  \
579            dimetindene       Launched   
4592  dimethindene-(S)-(+)    Preclinical   

                    